# PART2

## Import libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import os
import cv2
from scipy.spatial.distance import euclidean
from scipy.spatial.distance import cdist
import skimage
from skimage.io import imread
from skimage.color import rgb2gray
from skimage.feature import canny
from skimage.transform import hough_line, hough_line_peaks
from matplotlib import cm
from skimage.segmentation import slic, mark_boundaries
from skimage.color import label2rgb
from scipy.io import loadmat,savemat
import scipy.io
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from matplotlib.colors import ListedColormap

## SLIC

In [ ]:
class SLIC:
    def __init__(self,image_folder,image_names,n_segments,compactness,sigma,output_folder):
        self.image_folder=image_folder
        self.image_names=image_names
        self.n_segments=n_segments
        self.compactness=compactness
        self.sigma=sigma
        self.output_folder=output_folder
        
    def segment_images_with_slic(self):
        results = {}
        for image_name in self.image_names:
            image_path = os.path.join(self.image_folder, image_name)
            image = imread(image_path)
    
            segments = slic(image, n_segments=self.n_segments, compactness=self.compactness, sigma=self.sigma, start_label=1)
    
            boundaries = mark_boundaries(image, segments,(0, 0.5, 0.5))
            label_image_avg = label2rgb(segments, image, kind='avg')  # Optional: visualize average color within superpixels
            label_image_overlay=label2rgb(segments, image, kind='overlay')
            
            results[image_name] = {
                "original": image,
                "segments": segments,
                "boundaries": boundaries,
                "label_image": label_image_avg,
            }
    
            # Plot the results for this image
            fig, ax = plt.subplots(1, 4, figsize=(24, 8))
            ax[0].imshow(image)
            ax[0].set_title(f"Original Image: '{image_name}'")
            ax[0].axis("off")
    
            ax[1].imshow(boundaries)
            ax[1].set_title("Superpixel Boundaries")
            ax[1].axis("off")
    
            ax[2].imshow(label_image_avg)
            ax[2].set_title("Superpixels (Averaged Colors)")
            ax[2].axis("off")
            
            ax[3].imshow(label_image_overlay)
            ax[3].set_title("Superpixels (Overlayed Colors)")
            ax[3].axis("off")
    
            plt.tight_layout()
            plt.show()
        

        return results
    
        
    def save_segments_for_matlab(self):
        results=self.segment_images_with_slic()
            
        os.makedirs(self.output_folder, exist_ok=True)  # Ensure the output folder exists
    
        for image_name, data in results.items():
            segments = data['segments']  # Extract the segments matrix
            mat_file_name = os.path.join(self.output_folder, f"{os.path.splitext(image_name)[0]}_segments.mat")
            
            # Save to a MATLAB .mat file
            savemat(mat_file_name, {"segments": segments})
            print(f"Saved: {mat_file_name}")
            
        

In [ ]:
image_names = ["1.jpg","2.jpg","3.jpg","4.jpg","5.jpg","6.jpg","7.jpg","8.jpg","9.jpg","10.jpg"]
image_folder = "part2"
n_segments=200
compactness=10
sigma=1
output_folder = "part2/matlab_segments"
mark1=SLIC(image_folder=image_folder,image_names=image_names,n_segments=n_segments,compactness=compactness,sigma=sigma,output_folder=output_folder)
mark1.save_segments_for_matlab()

In [ ]:
image_names = ["1.jpg","2.jpg","3.jpg","4.jpg","5.jpg","6.jpg","7.jpg","8.jpg","9.jpg","10.jpg"]
image_folder = "part2"
n_segments=300
compactness=10
sigma=1
output_folder = "part2/matlab_segments"
mark1=SLIC(image_folder=image_folder,image_names=image_names,n_segments=n_segments,compactness=compactness,sigma=sigma,output_folder=output_folder)
mark1.save_segments_for_matlab()

In [ ]:
image_names = ["1.jpg","2.jpg","3.jpg","4.jpg","5.jpg","6.jpg","7.jpg","8.jpg","9.jpg","10.jpg"]
image_folder = "part2"
n_segments=400
compactness=10
sigma=1
output_folder = "part2/matlab_segments"
mark1=SLIC(image_folder=image_folder,image_names=image_names,n_segments=n_segments,compactness=compactness,sigma=sigma,output_folder=output_folder)
mark1.save_segments_for_matlab()

## K-MEAN

In [ ]:
class kmeans:
    def __init__(self,image_folder,segments_folder,feature_folder,cluster_result_folder,num_images,num_clusters):
        self.feature_folder=feature_folder
        self.segments_folder=segments_folder
        self.image_folder=image_folder
        self.cluster_result_folder=cluster_result_folder
        self.num_images=num_images
        self.num_clusters=num_clusters

    def load_feature_matrices(self):
        self.all_feature_matrices = []
        self.segment_sizes = []
        for img_idx in range(1, self.num_images + 1):
            file_path = os.path.join(self.feature_folder, f'feature_matrices_{img_idx}.mat')
            mat_data = scipy.io.loadmat(file_path)
            feature_matrix = mat_data['feature_matrix']
            self.all_feature_matrices.append(feature_matrix)
            self.segment_sizes.append(feature_matrix.shape[0])
            print(f"Loaded feature matrix for image {img_idx}: {feature_matrix.shape}")
        
    def perform_clustering(self):
        
        combined_features = np.vstack(self.all_feature_matrices)
        scaler = StandardScaler()
        self.standardized_features = scaler.fit_transform(combined_features)
        print(f"Combined feature matrix shape: {self.standardized_features.shape}")
        kmeans = KMeans(n_clusters=self.num_clusters, random_state=42)
        self.cluster_labels = kmeans.fit_predict(self.standardized_features)
        print(f"Clustering completed with {self.num_clusters} clusters.")
        print(self.cluster_labels.shape)

    def split_labels_func(self):
        self.split_labels = []
        start_idx = 0
        for size in self.segment_sizes:
            self.split_labels.append(self.cluster_labels[start_idx:start_idx + size])
            start_idx += size

    def visualize_clustering_with_boundaries(self):
        self.load_feature_matrices()
        self.perform_clustering()
        self.split_labels_func()
        os.makedirs(self.cluster_result_folder, exist_ok=True)
        for img_idx in range(1, self.num_images + 1):
           
            img_path = os.path.join(self.image_folder, f"{img_idx}.jpg")
            im = plt.imread(img_path)
            seg_path = os.path.join(self.segments_folder, f"{img_idx}_segments.mat")
            segments = scipy.io.loadmat(seg_path)['segments']

            unique_segments = np.unique(segments)
            cluster_map = np.zeros_like(segments, dtype=int)
            for segment_id, cluster_label in zip(unique_segments, self.split_labels[img_idx - 1]):
                cluster_map[segments == segment_id] = cluster_label


            # Create pseudo-color and overlay images
            pseudo_color_image = label2rgb(cluster_map, im, kind='overlay', bg_label=0)

            # Transparent overlay
            overlay = (0.6 * im + 0.4 * pseudo_color_image*255).astype(np.uint8)
            boundary_image = mark_boundaries(overlay, segments, color=(0, 0.3, 0.3))
            # Plot results
            plt.figure(figsize=(15, 5))
            plt.subplot(1, 3, 1)
            plt.imshow(im)
            plt.title(f"Original Image {img_idx}")
            plt.axis("off")

            plt.subplot(1, 3, 2)
            plt.imshow(pseudo_color_image)
            plt.title(f"K-Means Clustering Result {img_idx}")
            plt.axis("off")

            plt.subplot(1, 3, 3)
            plt.imshow(boundary_image)
            plt.title(f"Overlay with Clusters {img_idx}")
            plt.axis("off")
            plt.show()
            #output_path = os.path.join(self.cluster_result_folder, f"clustering_result_with_boundaries_{img_idx}.png")
            #plt.savefig(output_path)
            #plt.close()
            #print(f"Saved clustering result with boundaries for image {img_idx} to {self.cluster_result_folder}")
        

In [ ]:
image_folder = "part2"
segments_folder = "part2/matlab_segments"
feature_folder = "feature_matrices"
cluster_result_folder = "clustering_results"
num_images = 10
num_clusters = 4
k_mean_mark1=kmeans(image_folder,segments_folder,feature_folder,cluster_result_folder,num_images,num_clusters)
k_mean_mark1.visualize_clustering_with_boundaries()

In [ ]:
image_folder = "part2"
segments_folder = "part2/matlab_segments"
feature_folder = "feature_matrices"
cluster_result_folder = "clustering_results"
num_images = 10
num_clusters = 5
k_mean_mark1=kmeans(image_folder,segments_folder,feature_folder,cluster_result_folder,num_images,num_clusters)
k_mean_mark1.visualize_clustering_with_boundaries()

In [ ]:
image_folder = "part2"
segments_folder = "part2/matlab_segments"
feature_folder = "feature_matrices"
cluster_result_folder = "clustering_results"
num_images = 10
num_clusters = 6
k_mean_mark1=kmeans(image_folder,segments_folder,feature_folder,cluster_result_folder,num_images,num_clusters)
k_mean_mark1.visualize_clustering_with_boundaries()

# Contextual Clustering

In [ ]:
class ContextualClustering:
    def __init__(self, image_folder, segments_folder, feature_folder, output_folder, num_images, num_clusters, radius1, radius2, threshold=5):
        self.image_folder = image_folder
        self.segments_folder = segments_folder
        self.feature_folder = feature_folder
        self.output_folder = output_folder
        self.num_images = num_images
        self.num_clusters = num_clusters
        self.radius1 = radius1
        self.radius2 = radius2
        self.threshold = threshold

    def load_feature_matrices(self):
       
        self.all_feature_matrices = []
        self.segment_sizes = []
        for img_idx in range(1, self.num_images + 1):
            feature_path = os.path.join(self.feature_folder, f"feature_matrices_{img_idx}.mat")
            feature_matrix = scipy.io.loadmat(feature_path)['feature_matrix']
            self.all_feature_matrices.append(feature_matrix)
            self.segment_sizes.append(feature_matrix.shape[0])
            print(f"Loaded feature matrix for image {img_idx}: {feature_matrix.shape}")

    @staticmethod
    def compute_superpixel_centroids(segments):
        
        unique_segments = np.unique(segments)
        centroids = {}
        for segment_id in unique_segments:
            coords = np.argwhere(segments == segment_id)
            centroids[segment_id] = coords.mean(axis=0)  # Average row, col coordinates
        return centroids

    @staticmethod
    def find_neighbors(centroids, radius):
      
        segment_ids = list(centroids.keys())
        positions = np.array(list(centroids.values()))
        distances = cdist(positions, positions)  # Pairwise distances between centroids
        neighbors = {}
        for i, segment_id in enumerate(segment_ids):
            neighbors[segment_id] = [
                segment_ids[j] for j in range(len(segment_ids)) if 0 < distances[i, j] <= radius
            ]
        return neighbors

    def compute_contextual_features(self, feature_matrix, segments):
  
        centroids = self.compute_superpixel_centroids(segments)
        neighbors_level1 = self.find_neighbors(centroids, self.radius1)
        neighbors_level2 = self.find_neighbors(centroids, self.radius2)

        num_superpixels = feature_matrix.shape[0]
        num_features = feature_matrix.shape[1]
        contextual_features = np.zeros((num_superpixels, 3 * num_features))

        unique_segments = np.unique(segments)
        segment_to_idx = {seg: i for i, seg in enumerate(unique_segments)}

        for segment_id in unique_segments:
            idx = segment_to_idx[segment_id]
            original_feature = feature_matrix[idx]

            # First-level neighborhood average
            level1_neighbors = neighbors_level1[segment_id]
            level1_features = [
                feature_matrix[segment_to_idx[neighbor]]
                for neighbor in level1_neighbors
                if (segments == neighbor).sum() >= self.threshold
            ]
            level1_avg = np.mean(level1_features, axis=0) if level1_features else np.zeros_like(original_feature)

            # Second-level neighborhood average
            level2_neighbors = [
                neighbor
                for neighbor in neighbors_level2[segment_id]
                if neighbor not in level1_neighbors
            ]
            level2_features = [
                feature_matrix[segment_to_idx[neighbor]]
                for neighbor in level2_neighbors
                if (segments == neighbor).sum() >= self.threshold
            ]
            level2_avg = np.mean(level2_features, axis=0) if level2_features else np.zeros_like(original_feature)

            # Concatenate original, level1, and level2 features
            contextual_features[idx] = np.hstack([original_feature, level1_avg, level2_avg])

        return contextual_features

    def perform_clustering(self, combined_features):
     
        scaler = StandardScaler()
        standardized_features = scaler.fit_transform(combined_features)
        print(f"Combined feature matrix shape: {standardized_features.shape}")
        kmeans = KMeans(n_clusters=self.num_clusters, random_state=42)
        self.cluster_labels = kmeans.fit_predict(standardized_features)
        print(f"Clustering completed with {self.num_clusters} clusters.")

    def split_labels_func(self):
       
        self.split_labels = []
        start_idx = 0
        for size in self.segment_sizes:
            self.split_labels.append(self.cluster_labels[start_idx:start_idx + size])
            start_idx += size

    def visualize_clustering_with_boundaries(self):
    
        os.makedirs(self.output_folder, exist_ok=True)
        for img_idx in range(1, self.num_images + 1):
            img_path = os.path.join(self.image_folder, f"{img_idx}.jpg")
            im = plt.imread(img_path)
            seg_path = os.path.join(self.segments_folder, f"{img_idx}_segments.mat")
            segments = scipy.io.loadmat(seg_path)['segments']

            unique_segments = np.unique(segments)
            cluster_map = np.zeros_like(segments, dtype=int)
            for segment_id, cluster_label in zip(unique_segments, self.split_labels[img_idx - 1]):
                cluster_map[segments == segment_id] = cluster_label

            
            pseudo_color_image = label2rgb(cluster_map, im, kind='overlay', bg_label=0)

            # Transparent overlay
            overlay = (0.4 * im + 0.6 * pseudo_color_image*255).astype(np.uint8)
            boundraies = mark_boundaries(overlay, segments, color=(0, 0.5, 0.5))
            plt.figure(figsize=(12, 6))
            plt.subplot(1, 3, 1)
            plt.imshow(im)
            plt.title(f"Original Image {img_idx}")
            plt.axis("off")

            plt.subplot(1, 3, 2)
            plt.imshow(overlay)
            plt.title(f"Contextual Clustering {img_idx}")
            plt.axis("off")

            plt.subplot(1, 3, 3)
            plt.imshow(boundraies)
            plt.title(f"Boundaries with Context {img_idx}")
            plt.axis("off")
            plt.show()

    def contextual_clustering_pipeline(self):
        
        self.load_feature_matrices()

        all_contextual_features = []
        for img_idx in range(1, self.num_images + 1):
            segments_path = os.path.join(self.segments_folder, f"{img_idx}_segments.mat")
            segments = scipy.io.loadmat(segments_path)['segments']
            contextual_features = self.compute_contextual_features(self.all_feature_matrices[img_idx - 1], segments)
            all_contextual_features.append(contextual_features)

        combined_features = np.vstack(all_contextual_features)
        self.perform_clustering(combined_features)
        self.split_labels_func()
        self.visualize_clustering_with_boundaries()

In [ ]:
# Initialize and run the pipeline
contextual_clustering = ContextualClustering(
    image_folder="part2",
    segments_folder="part2/matlab_segments",
    feature_folder="feature_matrices",
    output_folder="contextual_clustering_results",
    num_images=10,
    num_clusters=5,  # Adjust as needed
    radius1=20*1.5,       # Adjust based on superpixel size
    radius2=20*2.5,       # Larger radius for second-level neighbors
    threshold=10      # Minimum pixels to include a superpixel in a ring
)

contextual_clustering.contextual_clustering_pipeline()